In [8]:
#!pip install --user faiss-cpu

In [1]:
import faiss
import numpy as np
import requests

### For a phrase - not saved in vector db

In [2]:
res = requests.post(url='http://localhost:11434/api/embeddings',
                    json={
                        'model':'llama3.2',
                        'prompt': 'Hello World!'
                        
                    })

In [3]:
print(res.json()['embedding'])

[-0.5786440968513489, 0.6307854652404785, -0.10577905923128128, -1.1337850093841553, 1.7348729372024536, 2.5147011280059814, 1.5284404754638672, -0.49296075105667114, -1.9213272333145142, 0.13850902020931244, -1.4749040603637695, 1.4097622632980347, 0.8826233148574829, 1.294507622718811, -0.9269030690193176, -0.5369409322738647, -1.186029314994812, -0.030699769034981728, -0.12573619186878204, -0.5334165692329407, -0.8619019389152527, -0.41534164547920227, 1.4017410278320312, -1.0536458492279053, 0.5242311954498291, -0.27939680218696594, 0.09333855658769608, -1.1475945711135864, 2.1926541328430176, 1.5979654788970947, -0.9729374647140503, -0.043138206005096436, 1.4747675657272339, 0.20132358372211456, 1.0249637365341187, -0.5834943652153015, 0.6232419610023499, 1.7729202508926392, -0.9898797273635864, -1.027977466583252, 0.29012948274612427, 0.12283812463283539, 0.12169996649026871, -0.01878688670694828, -1.4182080030441284, -1.1645125150680542, -1.4841811656951904, 0.7907684445381165, 

In [5]:
print(len(res.json()['embedding']))

3072


### For a Bunch of sentences and save in faiss

In [17]:
d = 3072 # dimension of embedding vector returned by llama3.2

In [13]:
titles = [
    'To Kill a Mockingbird',
    'Pride and Prejudice',
    'The Great Gatsby',
    '1984',
    'The Lord of the Rings'
 ]

In [18]:
# first create a vector database index

index = faiss.IndexFlatL2(d)

In [19]:
# create array of len same number of titles and dimension as d, this will contain all the vectors eventually, 1 vector per title

X = np.zeros(shape=(len(titles),d), dtype='float32')

# mentioning dtype is mandatory else it will not be able to interpret d value properly

In [20]:
for i, title in enumerate(titles):
    res = requests.post(url='http://localhost:11434/api/embeddings',
                    json={
                        'model':'llama3.2',
                        'prompt': title
                        
                    })
    embedding = res.json()['embedding'] # save embedding in a variable and add to X
    
    X[i] = np.array(embedding)

In [21]:
index.add(X) # adding to database

In [30]:
new_prompt = 'The Fountainhead' #'The Wheel of Time' # prompt to perform similarity search

In [31]:
res = requests.post(url='http://localhost:11434/api/embeddings',
                json={
                    'model':'llama3.2',
                    'prompt': new_prompt

                })
embedding = np.array([res.json()['embedding']], dtype ='float32' )# save embedding in a variable and add to X

In [32]:
# calculating similarity, D=distance, I=Index

D, I = index.search(embedding, 5)

In [33]:
print(np.array(titles)[I.flatten()])

['The Great Gatsby' 'Pride and Prejudice' 'To Kill a Mockingbird'
 'The Lord of the Rings' '1984']
